In [13]:
import os

print(os.getcwd())

C:\Users\gopij


In [22]:
print(os.listdir('.'))
#help(os.listdir)
'numExc' in os.listdir('.')

['.android', '.bash_history', '.cache', '.config', '.eclipse', '.git', '.gitconfig', '.gradle', '.ipynb_checkpoints', '.ipython', '.jupyter', '.lemminx', '.lesshst', '.m2', '.rediscli_history', '.ssh', '.sts4', '.viminfo', '.vscode', 'AppData', 'Application Data', 'Contacts', 'Cookies', 'demo', 'Documents', 'Downloads', 'Favorites', 'git', 'IBMDS.ipynb', 'Links', 'Local Settings', 'Music', 'My Documents', 'NetHood', 'netstat', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TM.blf', 'NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'OneDrive', 'Postman', 'PrintHood', 'PycharmProjects', 'Recent', 'Saved Games', 'Searches', 'SendTo', 'Start Menu', 'Templates', 'Videos']


False